# Import libraries

In [23]:
import pandas as pd
import numpy as np 
import datetime
# import time
# import sys
# from scipy import stats
from statsmodels.stats import stattools

# Include py file

In [53]:
## to remove the directory and its contents
# !rmdir /s /q RiskLabAI.py

## to clone the RiskLabAI directory
!git clone https://github.com/RiskLabAI/RiskLabAI.py.git

In [50]:
import sys
sys.path.append('./RiskLabAI.py/')
from RiskLabAI import *

In [52]:
import RiskLabAI as risklab

ImportError: cannot import name 'generate_time_bar_dataframe' from 'RiskLabAI.data.structures.standard_bars' (c:\Users\hamid\OneDrive\GitHub\Notebooks.py\./RiskLabAI.py\RiskLabAI\data\structures\standard_bars.py)

In [43]:

# from RiskLabAI import *

# Import Tick data

In [28]:
dir = "https://raw.githubusercontent.com/risk-labratory/data/main/"
url = dir + "IVE_2020.csv"
dataframe = pd.read_csv(url, header = 0)
dataframe['dates'] = pd.to_datetime(dataframe['dates'])
dataframe.set_index('dates', inplace = True, drop = True)
dataframe.drop_duplicates(inplace = True)
dataframe = dataframe[(dataframe.index.hour >= 9) & (dataframe.index.hour<16)]
dataframe.head()

,price,bid,ask,size
dates,,,,
2020-01-02 09:30:00,130.68,130.59,130.60,20625
2020-01-02 09:30:01,130.50,130.50,130.77,200
2020-01-02 09:30:04,130.53,130.52,130.78,100
2020-01-02 09:30:04,130.55,130.52,130.78,100
2020-01-02 09:30:04,130.53,130.52,130.78,200


In [49]:
from RiskLabAI import generate_time_bar_dataframe

ImportError: cannot import name 'generate_time_bar_dataframe' from 'RiskLabAI' (c:\Users\hamid\OneDrive\GitHub\Notebooks.py\./RiskLabAI.py\RiskLabAI\__init__.py)

In [38]:
import sys
sys.path.append('./RiskLabAI.py/')

In [41]:
from RiskLabAI import generate_time_bar_dataframe
ohlcvTime = generate_time_bar_dataframe(dataframe, frequency = "30Min")
ohlcvTime.head(10)

ImportError: cannot import name 'generate_time_bar_dataframe' from 'RiskLabAI' (c:\Users\hamid\OneDrive\GitHub\Notebooks.py\./RiskLabAI.py\RiskLabAI\__init__.py)

In [ ]:
timeBarLen = ohlcvTime.shape[0]
ohlcvTick = generate_tick_bar_dataframe(dataframe, tickPerBar = None, numberBars = timeBarLen)
ohlcvTick.head(10)

In [ ]:
ohlcvVolume = generate_volume_bar_dataframe(dataframe, volumePerBar = None, numberBars = timeBarLen)
ohlcvVolume.head(10)

In [ ]:
ohlcvDollar = dollarBar(dataframe, dollarPerBar = None, numberBars = timeBarLen)
ohlcvDollar.head(10)

In [ ]:
countAverage = pd.DataFrame()
countAverage['Time'] = ohlcvTime.resample("1W")['TickCount'].mean()
countAverage['Tick'] = ohlcvTick.resample("1W")['TickCount'].mean()
countAverage['Volume'] = ohlcvVolume.resample("1W")['TickCount'].mean()
countAverage['Dollar'] = ohlcvDollar.resample("1W")['TickCount'].mean()
countAverage

In [ ]:
countAverage

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Time, name='Time',))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Tick, name='Tick',))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Volume, name='Volume',))
fig.add_trace(go.Scatter(x=countAverage.index, y=countAverage.Dollar, name='Dollar',))


fig.update_layout(
    title="Bagging Classifier Accuracy (function of p and N)",
    xaxis_title="p (The accuracy of a classifier is the probability p of labeling a prediction as 1)",
    yaxis_title="N (The number of independent classifers)",
    template="plotly_dark",
)
# Edit the layout
fig.update_layout(title='Standard Bars Frequency',
                   xaxis_title='Date',
                   yaxis_title='Frequency')


fig.write_json('Standard Bars Frequency.json')

fig.show()

In [ ]:
# Select a smaller range
startDate = datetime.datetime(2020, 3, 1)
endDate = datetime.datetime(2020, 3, 5)
dataframe2 = dataframe[((dataframe.index >= startDate) & (dataframe.index <= endDate))][['price', 'size']]
dataframe2.drop_duplicates()
dataframe2['ret'] = np.log(dataframe2['price']) - np.log(dataframe2['price'].shift(1))
dataframe2['label'] = np.sign(dataframe2['ret'])
dataframe2['volumelabeled'] = dataframe2['label']*dataframe2['size']
dataframe2['dollarslabeled'] = dataframe2['volumelabeled']*dataframe2['price']
dataframe2.dropna(inplace = True)
dataframe2.head()

In [ ]:
ohlcv_time = timeBar(dataframe2, frequency = "30Min")
ohlcv_tick = tickBar(dataframe2, tickPerBar = 1000, numberBars = None)
ohlcv_volume = volumeBar(dataframe2, volumePerBar = 10000, numberBars = None)
ohlcv_dollar = dollarBar(dataframe2, dollarPerBar = 500000, numberBars = None)

In [ ]:
ohlcvinfo, thetasabsolute, thresholds = infoBar(dataframe2, type = "dollar", tickExpectedInit = 500)